In [19]:
import sqlite3
import pandas as pd

In [20]:
conn = sqlite3.connect('cc1.sqlite')

In [21]:
curs = conn.cursor()

**Q10:** Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members

In [22]:
r1 = curs.execute("""SELECT 
                    facility,
                    SUM(revenue) AS total_revenue
                    FROM (
                        SELECT 
                            f.name as facility,
                            (CASE WHEN b.memid = 0 THEN b.slots*f.guestcost
                                  WHEN b.memid != 0 THEN b.slots*f.membercost
                                  END) AS revenue
                        FROM Facilities AS f
                        INNER JOIN Bookings AS b
                        USING(facid)) AS facility_revenue
                    GROUP BY facility
                    HAVING total_revenue < 1000
                    ORDER BY total_revenue""")

In [23]:
print(pd.DataFrame(r1.fetchall(), columns=['facility', 'total_revenue']))

        facility  total_revenue
0   Table Tennis            180
1  Snooker Table            240
2     Pool Table            270


**Q11:** Produce a report of members and who recommended them in alphabetic surname,firstname order

In [27]:
r2 = curs.execute("""SELECT
                        surname,
                        firstname,
                        recommendedby
                    FROM Members
                    ORDER BY surname, firstname""")

In [28]:
print(pd.DataFrame(r2.fetchall(), columns=['surname', 'firstname', 'recommendedby']))

              surname  firstname recommendedby
0               Bader   Florence             9
1               Baker       Anne             9
2               Baker    Timothy            13
3              Boothe        Tim             3
4             Butters     Gerald             1
5              Coplin       Joan            16
6             Crumpet      Erica             2
7                Dare      Nancy             4
8             Farrell      David              
9             Farrell     Jemima              
10              GUEST      GUEST              
11            Genting    Matthew             5
12               Hunt       John            30
13              Jones      David             4
14              Jones    Douglas            11
15           Joplette     Janice             1
16          Mackenzie       Anna             1
17               Owen    Charles             1
18             Pinker      David            13
19            Purview  Millicent             2
20           

**Q12:** Find the facilities with their usage by member, but not guests

In [29]:
r3 = curs.execute("""SELECT
                        f.name AS facility,
                        SUM(b.slots) as total_member_bookings
                    FROM Members AS m
                    INNER JOIN Bookings AS b
                    USING(memid)
                    INNER JOIN Facilities AS f
                    USING(facid)
                    WHERE b.memid != 0
                    GROUP BY facility
                    ORDER BY total_member_bookings DESC""")

In [30]:
print(pd.DataFrame(r3.fetchall(), columns=['facility', 'total_member_bookings']))

          facility  total_member_bookings
0  Badminton Court                   1086
1   Tennis Court 1                    957
2   Massage Room 1                    884
3   Tennis Court 2                    882
4    Snooker Table                    860
5       Pool Table                    856
6     Table Tennis                    794
7     Squash Court                    418
8   Massage Room 2                     54


**Q13:** Find the facilities usage by month, but not guests

In [33]:
r4 = curs.execute("""SELECT
                        f.name AS facility,
                        strftime('%m', starttime) AS month,
                        SUM(b.slots) as total_monthly_bookings
                    FROM Bookings AS b
                    INNER JOIN Facilities AS f
                    USING(facid)
                    WHERE b.memid != 0
                    GROUP BY facility, month
                    ORDER BY facility, month""")

In [34]:
print(pd.DataFrame(r4.fetchall(), columns=['facility', 'month', 'total_monthly_bookings']))

           facility month  total_monthly_bookings
0   Badminton Court    07                     165
1   Badminton Court    08                     414
2   Badminton Court    09                     507
3    Massage Room 1    07                     166
4    Massage Room 1    08                     316
5    Massage Room 1    09                     402
6    Massage Room 2    07                       8
7    Massage Room 2    08                      18
8    Massage Room 2    09                      28
9        Pool Table    07                     110
10       Pool Table    08                     303
11       Pool Table    09                     443
12    Snooker Table    07                     140
13    Snooker Table    08                     316
14    Snooker Table    09                     404
15     Squash Court    07                      50
16     Squash Court    08                     184
17     Squash Court    09                     184
18     Table Tennis    07                      98
